In [1]:
%pylab inline
from tqdm import tqdm_notebook
import os.path

from rd_model import RazzleDazzleGame, Position
from rd_bots import PositionBot, PositionNet, IdiotBot
from rd_helpers import *

from multiprocessing import Pool

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import random
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
def correct_predictions(output,actual):
    return np.sum(1-np.floor(np.abs(output.flatten()-actual.flatten())))

def generate_game(bot1,bot2):
    game = RazzleDazzleGame(bot1, bot2)
    game.play()
    return game

def get_game_data(game_str):
    gd=[]
    if game_str.strip()[-1] == '8':  # get value
        v = 1
    else:
        v = -1
    pos = Position(score=v)
    white_move = True
    for ms in game_str.split(','):
        m = pos.parse_move(ms)
        if not white_move:
            m = pos.rotate_move(m)

        gd.append([pos.state.flatten(), pos.get_piece_on_square(m[0]), m[-1], v])
        pos=pos.move(m)
        pos.rotate()
    
    return gd

def generate_games(bot1,bot2, games, fname, path):
    game_lengths=[]
    wins=0
    losses=0
    print "Generating %d games of %s vs. %s, stored at %s" % (games, bot1.name,bot2.name, os.path.join(path,fname))
    games=[pool.apply_async(generate_game, args=(bot1,bot2)) for ii in range(games)]
    for game in tqdm_notebook(games):
        game=game.get()
        game_lengths.append(len(game))
        if game.winner>0:
            wins+=1
        else:
            losses+=1
        with open(os.path.join(path,fname),'a') as f:
            f.write(str(game)+'\n')
    print "Stats: mean game length = %d, win/loss = %d / %d" % (np.mean(game_lengths),wins,losses)

            
def generate_dataset_old(prefix,path, test_size, moves_before_end):
    fname=os.path.join(path,prefix + ".txt")
    wins=[]
    losses=[]
    
    gds=[]
    with open(fname,'r') as f:
        gds = [pool.apply_async(get_game_data,args=(gs,)) for gs in f]

    for g in tqdm_notebook(gds):
        g=g.get()
        for p in g[-moves_before_end:]:
            if p[-1]>0:
                wins.append(p)
            else:
                losses.append(p)

    num_scores=min(len(wins),len(losses))
    test_dataset=wins[:test_size]+losses[:test_size]
    dataset=wins[test_size:num_scores]+losses[test_size:num_scores]
    random.shuffle(dataset)
    random.shuffle(test_dataset)
    dataset = np.transpose(dataset)
    test_dataset = np.transpose(test_dataset)
    positions=np.vstack(dataset[0])
    test_positions=np.vstack(test_dataset[0])
    scores=np.array(dataset[-1],dtype=float)
    test_scores=np.array(test_dataset[-1],dtype=float)

    np.savez_compressed(os.path.join(path,prefix+"_dataset.npz"), 
                        positions=positions, scores=scores, 
                        test_positions=test_positions,test_scores=test_scores)
    
    return (positions,scores,test_positions,test_scores)

def get_game_data2(game_str):
    gd=[]
    if game_str.strip()[-1] == '8':  # get value
        v = 1
    else:
        v = -1
    pos = Position(score=v)
    white_move = True
    for ms in game_str.split(','):
        m = pos.parse_move(ms)
        if not white_move:
            m = pos.rotate_move(m)

        gd.append(pos.state.flatten())
        pos=pos.move(m)
        pos.rotate()
    return gd,v

def generate_dataset(prefix,path, test_size, moves_before_end):
    fname=os.path.join(path,prefix + ".txt")
    wins=[]
    losses=[]
    
    gds=[]
    with open(fname,'r') as f:
        gds = [pool.apply_async(get_game_data2,args=(gs,)) for gs in f]

    for g in tqdm_notebook(gds):
        g=g.get()
        gd=g[0]
        v=g[-1]
        if v>0:
            wins.append((np.array(gd[-moves_before_end:]),v))
        else:
            losses.append((np.array(gd[-moves_before_end:]),v))

    num_scores=min(len(wins),len(losses))
    test_dataset=np.array(wins[:test_size]+losses[:test_size])
    dataset=np.array(wins[test_size:num_scores]+losses[test_size:num_scores])

    random.shuffle(dataset)
    random.shuffle(test_dataset)
#     dataset = np.transpose(dataset)
#     test_dataset = np.transpose(test_dataset)
#     positions=np.vstack(dataset[0])
#     test_positions=np.vstack(test_dataset[0])
#     scores=np.array(dataset[-1],dtype=float)
#     test_scores=np.array(test_dataset[-1],dtype=float)

    np.savez_compressed(os.path.join(path,prefix+"_dataset.npz"), 
                        dataset=dataset, test_dataset=test_dataset)
    
    return dataset,test_dataset


pool=Pool(processes=8)

In [3]:
def train_position_iteration(name,datapath,epochs,lr=0.001,momentum=0.5,log_interval=10,cuda=False):

    index=current_file_index(datapath,prefix=name)
    fname = get_current_filename(datapath=datapath, prefix=name, suffix='.bot')
    fname_out = get_next_filename(datapath=datapath, prefix=name, suffix='.bot')

    with np.load(os.path.join(datapath,fname+'_dataset.npz')) as f:
        #positions,scores,test_positions,test_scores=f['positions'],f['scores'],f['test_positions'],f['test_scores']
        dataset,test_dataset = f['dataset'], f['test_dataset']

    bot1 = PositionBot(fname,path=datapath)
    model = bot1.posnet
    print "Training: " + fname
    print "Training size: %d\t Test size: %d" % (len(dataset),len(test_dataset))

    loss_fn=nn.MSELoss()

    if cuda:
        model.cuda()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)


    #test data/value
    #test_data  = torch.from_numpy(test_positions).float()
    #test_value = torch.from_numpy(test_scores).float()
    #if cuda:
    #    test_data, test_value = test_data.cuda(), test_value.cuda()            
    #test_data, test_value = Variable(test_data), Variable(test_value)
            
    for epoch in range(epochs):
        model.train()
        total=0
        correct=0
        for ii, data in enumerate(dataset):
            #print data[0].shape
            score = data[-1]
            data  = torch.from_numpy(data[0]).float()
#            value = torch.from_numpy(scores[ii*batch_size:(ii+1)*batch_size]).float()
#            if cuda:
#                data, value = data.cuda(), value.cuda()            
#            data, value = Variable(data), Variable(value)
            data = Variable(data)
            optimizer.zero_grad()
            output = model(data)
            #shift output by one slot and make last estimation who won
            value = output.data.cpu().numpy().copy()
            value[:-1]=value[1:]
            value[-1]=score
            value = torch.from_numpy(value).float()               
            value = Variable(value)
            
            #print "output:", output
            #print "value:", value
            loss = loss_fn(output, value)
            loss.backward()
            optimizer.step()

            #correct+=correct_predictions(output.data.cpu().numpy(),value.data.cpu().numpy())
            #total+=len(value.data.cpu().numpy())

        if epoch % log_interval == log_interval-1:
            #print "Train:\t%.3f%% correct\tLoss: %.6f\t%d/%d" % (correct*100./total,loss.data[0],epoch+1,epochs)
            print "Train Loss: %.6f\t%d/%d" % (loss.data[0],epoch+1,epochs)

#         model.eval()
#         test_output=model(test_data)
#         value = output.data.cpu().numpy().copy()
#         value[:-1]=value[1:]
#         value[-1]=score
#         value = torch.from_numpy(value).float()               
#         value = Variable(value)
#         test_loss=loss_fn(test_output,test_value)
#         #correct=correct_predictions(test_output.data.cpu().numpy(),test_value.data.cpu().numpy())
#         total=len(test_value.data.cpu().numpy())    
        
#         if epoch % log_interval == log_interval-1:
#             print "Test:\t%.3f%% correct\tLoss: %.6f" % (correct*100./total,test_loss.data[0])
    
    
    print "Saving trained bot as: " + fname_out
    torch.save(model.state_dict(), os.path.join(datapath,fname_out))
    
    #return test_output,test_value

In [4]:
games=25000
name='medium'
path='data/'

# Generate some seed data
ib1=IdiotBot()
ib2=IdiotBot()
generate_games(ib1,ib2,games,name+'.txt',path)
#(positions,scores,test_positions,test_scores)=generate_dataset(name,'data/',test_size=1000, moves_before_end=10)
#train_position_iteration(name=name,datapath='data/',batch_size=200,epochs=25,lr=0.01,momentum=0.5,log_interval=5,cuda=True)


Generating 25000 games of Idiot vs. Idiot, stored at data/medium.txt


Widget Javascript not detected.  It may not be installed or enabled properly.



Stats: mean game length = 223, win/loss = 11697 / 13303


In [6]:
generate_dataset??

In [3]:
games=10000
name='medium'
path='data/'
dataset,test_dataset=generate_dataset(name,'data/',test_size=100, moves_before_end=10)

KeyboardInterrupt: 

In [8]:
train_position_iteration(name=name,datapath='data/',epochs=5,lr=0.01,momentum=0.5,log_interval=1,cuda=False)

Failed to load: data/00000_medium.bot
Loading new PositionNet model
Training: 00000_medium.bot
Training size: 23194	 Test size: 200
Train Loss: 0.078958	1/5
Train Loss: 0.045387	2/5
Train Loss: 0.023024	3/5
Train Loss: 0.012217	4/5
Train Loss: 0.032637	5/5
Saving trained bot as: 00000_medium.bot


In [9]:
games=100
name='iter'
path='data/'
fname = get_current_filename(datapath='data/', prefix=name, suffix='.bot')
b1 = PositionBot(fname,path='data/')
b2 = PositionBot(fname,path='data/')
generate_games(b1,IdiotBot(),games,fname+'.txt',path)


Generating 100 games of 00220_iter.bot vs. Idiot, stored at data/00220_iter.bot.txt



Stats: mean game length = 271, win/loss = 40 / 60


In [8]:
games=200
name='iter'
path='data/'
for ii in range(100):
    fname = get_current_filename(datapath='data/', prefix=name, suffix='.bot')
    b1 = PositionBot(fname,path='data/')
    b2 = PositionBot(fname,path='data/')
    generate_games(b1,b2,games,fname+'.txt',path)
    dataset,test_dataset=generate_dataset(fname,'data/',test_size=10, moves_before_end=100)
    train_position_iteration(name=name,datapath='data/',epochs=10,lr=0.001,momentum=0.5,log_interval=1,cuda=False)

Generating 200 games of 00120_iter.bot vs. 00120_iter.bot, stored at data/00120_iter.bot.txt



Stats: mean game length = 545, win/loss = 103 / 97



Training: 00120_iter.bot
Training size: 174	 Test size: 20
Train Loss: 0.002023	1/10
Train Loss: 0.002030	2/10
Train Loss: 0.002038	3/10
Train Loss: 0.002045	4/10
Train Loss: 0.002052	5/10
Train Loss: 0.002059	6/10
Train Loss: 0.002066	7/10
Train Loss: 0.002073	8/10
Train Loss: 0.002079	9/10
Train Loss: 0.002086	10/10
Saving trained bot as: 00121_iter.bot
Generating 200 games of 00121_iter.bot vs. 00121_iter.bot, stored at data/00121_iter.bot.txt



Stats: mean game length = 578, win/loss = 109 / 91



Training: 00121_iter.bot
Training size: 162	 Test size: 20
Train Loss: 0.028157	1/10
Train Loss: 0.028161	2/10
Train Loss: 0.028164	3/10
Train Loss: 0.028167	4/10
Train Loss: 0.028170	5/10
Train Loss: 0.028173	6/10
Train Loss: 0.028175	7/10
Train Loss: 0.028178	8/10
Train Loss: 0.028181	9/10
Train Loss: 0.028184	10/10
Saving trained bot as: 00122_iter.bot
Generating 200 games of 00122_iter.bot vs. 00122_iter.bot, stored at data/00122_iter.bot.txt


...


Training: 00217_iter.bot
Training size: 174	 Test size: 20
Train Loss: 0.028271	1/10
Train Loss: 0.028264	2/10
Train Loss: 0.028258	3/10
Train Loss: 0.028251	4/10
Train Loss: 0.028245	5/10
Train Loss: 0.028239	6/10
Train Loss: 0.028232	7/10
Train Loss: 0.028226	8/10
Train Loss: 0.028220	9/10
Train Loss: 0.028213	10/10
Saving trained bot as: 00218_iter.bot
Generating 200 games of 00218_iter.bot vs. 00218_iter.bot, stored at data/00218_iter.bot.txt



Stats: mean game length = 452, win/loss = 110 / 90



Training: 00218_iter.bot
Training size: 160	 Test size: 20
Train Loss: 0.001093	1/10
Train Loss: 0.001092	2/10
Train Loss: 0.001090	3/10
Train Loss: 0.001089	4/10
Train Loss: 0.001088	5/10
Train Loss: 0.001086	6/10
Train Loss: 0.001085	7/10
Train Loss: 0.001083	8/10
Train Loss: 0.001082	9/10
Train Loss: 0.001081	10/10
Saving trained bot as: 00219_iter.bot
Generating 200 games of 00219_iter.bot vs. 00219_iter.bot, stored at data/00219_iter.bot.txt



Stats: mean game length = 492, win/loss = 100 / 100



Training: 00219_iter.bot
Training size: 180	 Test size: 20
Train Loss: 0.000976	1/10
Train Loss: 0.000977	2/10
Train Loss: 0.000978	3/10
Train Loss: 0.000978	4/10
Train Loss: 0.000979	5/10
Train Loss: 0.000980	6/10
Train Loss: 0.000981	7/10
Train Loss: 0.000981	8/10
Train Loss: 0.000982	9/10
Train Loss: 0.000983	10/10
Saving trained bot as: 00220_iter.bot


In [6]:
fname

'00000_iter.bot'

In [8]:
fname[:-4]

'00000_iter'